In [ ]:
from sympy import sympify, limit, oo

This is the `1s0 -> 1s0` and `3s1 -> 3s1`  transition

In [ ]:
ss = """
L ** 2 * (L ** 2 + k ** 2 / 4 - k * p_o - p_i ** 2 + p_o ** 2) * exp(
    -(p_i ** 2 + (k / 2 - p_o) ** 2) / L ** 2
) / (pi ** 2 * k * p_o) - L ** 2 * (
    L ** 2 + k ** 2 / 4 + k * p_o - p_i ** 2 + p_o ** 2
) * exp(
    -(p_i ** 2 + (k / 2 + p_o) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_o
) + L ** 2 * (
    L ** 2 + k ** 2 / 4 - k * p_i + p_i ** 2 - p_o ** 2
) * exp(
    2 * k * p_i / L ** 2 - (p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_i
) - L ** 2 * (
    L ** 2 + k ** 2 / 4 + k * p_i + p_i ** 2 - p_o ** 2
) * exp(
    -(p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2
) / (
    pi ** 2 * k * p_i
)
"""
ss = sympify(ss)

In [ ]:
ss

In [ ]:
limit(ss, "L", oo)

This is the `3s1 -> 3d1` transition Apparently needs a factor of two from `d->s`.

In [ ]:
sd = """
2 * sqrt(2) * L ** 2 * p_o ** 2 * exp(-(p_o ** 2 + (k / 2 + p_i) ** 2) / L ** 2) / (
    3 * pi ** 2 * k * p_i
) - 2 * sqrt(2) * L ** 2 * p_o ** 2 * exp(-(p_o ** 2 + (k / 2 - p_i) ** 2) / L ** 2) / (
    3 * pi ** 2 * k * p_i
) + (
    -sqrt(2) * L ** 6 / (2 * pi ** 2 * k * p_o ** 3)
    - sqrt(2) * L ** 4 / (2 * pi ** 2 * p_o ** 2)
    - 2 * sqrt(2) * L ** 4 / (3 * pi ** 2 * k * p_o)
    - sqrt(2) * L ** 2 * k / (6 * pi ** 2 * p_o)
    - 2 * sqrt(2) * L ** 2 / (3 * pi ** 2)
    - 2 * sqrt(2) * L ** 2 * p_o / (3 * pi ** 2 * k)
) * exp(
    -(p_i ** 2 + (k / 2 + p_o) ** 2) / L ** 2
) + (
    sqrt(2) * L ** 6 / (2 * pi ** 2 * k * p_o ** 3)
    - sqrt(2) * L ** 4 / (2 * pi ** 2 * p_o ** 2)
    + 2 * sqrt(2) * L ** 4 / (3 * pi ** 2 * k * p_o)
    + sqrt(2) * L ** 2 * k / (6 * pi ** 2 * p_o)
    - 2 * sqrt(2) * L ** 2 / (3 * pi ** 2)
    + 2 * sqrt(2) * L ** 2 * p_o / (3 * pi ** 2 * k)
) * exp(
    -(p_i ** 2 + (k / 2 - p_o) ** 2) / L ** 2
)
"""
sd = sympify(sd)

In [ ]:
ds = sd.subs({"p_i": "p_o", "p_o": "p_i"}) * 2

In [ ]:
str(ds)

In [ ]:
sd.simplify()

In [ ]:
limit(sd, "L", oo)

In [ ]:
from numpwd.qchannels.lsj import get_two_n_channels, generate_matrix_channels
from pandas import DataFrame

In [ ]:
channels = DataFrame(get_two_n_channels(l_max=2, s_max=1))
allowed = {
    "1s0": "s==l==j==0",
    "3s1": "(s==j==1 and l==0)",
    "3d1": "(s==j==1 and l==2)",
}

channels = channels.query(" or ".join(allowed.values())).reset_index()
channels

In [ ]:
allowed_transitions = [
    lambda ci, co: ci["ml"] == co["ml"]
    and ci["ms"] == co["ms"]
    and ci["mj"] == co["mj"]
    and ci["j"] == co["j"]
    and ci["s"] == co["s"]
    and ci["l"] in [0, 2]
    and co["l"] in [0, 2]
    and not (ci["l"] == co["l"] == 2)
]
op_channels = (
    DataFrame(
        list(
            generate_matrix_channels(
                channels.to_dict("records"), allowed_transitions=allowed_transitions
            )
        )
    )
    .drop(
        columns=["cg_o", "cg_i", "index_o", "index_i", "ml_o", "ml_i", "ms_o", "ms_i"]
    )
    .groupby(["l_o", "s_i", "j_i", "l_i", "s_o", "j_o", "mj_i", "mj_o"])
    .sum()
    .reset_index()
)
print(op_channels)

## Load in density

In [ ]:
import os
import re
from numpwd.densities.h5 import read_h5

In [ ]:
DATA = os.environ["DATA"]
DENS_DIR = os.path.join(DATA, "4he", "chsms")
DENSITIES = [
    el
    for el in os.listdir(DENS_DIR)
    if el.endswith(".h5") and "th=1.80E+02" in el and "cut=4" in el
]

In [ ]:
DENSITIES[0]

In [ ]:
patterns = (
    r"compton-dens-(?P<nuc>[0-9A-z]+)",
    r"\-(?P<potential>[a-z0-9\+]+)",
    r"\-(?P<order>[a-z0-9\+]+)",
    r"\-cut=(?P<cut>[0-9]+)",
    r"\-(?P<empot>[a-zA-Z]+)",
    r"\-(?P<cmpi>(?:[a-z0-9]+))",
    r"\-Lam=(?P<Lam>(?:[\.0-9]+))",
    r"\-c1=(?P<c1>(?:[\-\.0-9]+))",
    r"\-c3=(?P<c3>(?:[\-\.0-9]+))",
    r"\-c4=(?P<c4>(?:[\-\.0-9]+))",
    r"\-Lamnum=(?P<lambda>(?:[0-9\.e\+]+))",
    r"\-tnfcut=(?P<tnfcut>(?:[0-9]+))",
    r"\-om=(?P<omega>(?:[0-9\.]+E[\+\-][0-9]+))",
    r"\-th=(?P<theta>(?:[0-9\.E\+]+))",
    r"\-nx=(?P<nx>(?:[0-9]+))",
    r"\-nphi=(?P<nphi>(?:[0-9]+))",
    r"\-np12\=np34\=(?P<np12_np34>(?:[0-9\+]+))",
    r"\-np3\=(?P<np3>(?:[0-9\+]+))",
    r"\-nq4\=nq=(?P<nq4_nq>(?:[0-9\+]+))",
    r"\-j12max=(?P<j12max>(?:[0-9]+))",
    r"\-lmax=(?P<lmax>(?:[0-9]+))",
    r"\-lsummax=(?P<lsummax>(?:[0-9]+))",
    r"\-tau4max=(?P<tau4max>(?:[0-9]+))",
    r"\-rho\.h5",
)
ppat = ""
data = None
for pat in patterns:
    ppat += pat
    match = re.search(ppat, DENSITIES[0])
    if not match:
        print(data)
        raise ValueError(ppat)
    else:
        data = match.groupdict()

In [ ]:
pattern = re.compile("".join(patterns))
pattern.search(DENSITIES[0]).groupdict()

In [ ]:
dtypes = {
    int: ["nx", "nphi", "j12max", "lmax", "lsummax", "tau4max", "cut"],
    float: ["lambda", "omega", "theta", "c1", "c3", "c4", "Lam"],
}

densities = DataFrame([pattern.search(el).groupdict() for el in DENSITIES])
for dtype, cols in dtypes.items():
    for col in cols:
        densities[col] = densities[col].astype(dtype)

densities["file"] = DENSITIES

densities = densities.sort_values("omega").reset_index(drop=True)
densities.drop(columns=["file"]).head()

In [ ]:
filename = DENSITIES[0]
dens = read_h5(os.path.join(DENS_DIR, filename))
dens

In [ ]:
HBARC = 197

## Convert to operator format

In [ ]:
from numpwd.operators.base import Operator
from numpwd.integrate.numeric import ExpressionMap
from numpwd.convolution.channels import get_channel_overlap_indices
import numpy as np

In [ ]:
def allocate_matrix(mss, msd, mds, op_channels):
    matrix = []
    for _, channel in op_channels.iterrows():
        lo = "s" if channel["l_o"] == 0 else "d"
        li = "s" if channel["l_i"] == 0 else "d"
        so = str(2 * channel["s_o"] + 1)
        si = str(2 * channel["s_i"] + 1)
        jo = str(channel["j_o"])
        ji = str(channel["j_i"])

        ci = si + li + ji
        co = so + lo + jo

        if ci == co == "1s0" or ci == co == "3s1":
            matrix.append(mss * float(channel["cg"]))
        elif ci == "3s1" and co == "3d1":
            matrix.append(msd * float(channel["cg"]))
        elif ci == "3d1" and co == "3s1":
            matrix.append(mds * float(channel["cg"]))
        else:
            raise ValueError(f"ci={ci}, co={co}")

    return np.array(matrix)

In [ ]:
def allocate_op(mss, msd, mds, op_channels, p):
    op = Operator()
    op.channels = op_channels.drop(columns="cg")
    op.args = [("p_o", p), ("p_i", p)]
    op.isospin = {(0, 0, 0, 0): 1, (1, 1, 1, 1): 1, (1, 0, 1, 0): 1, (1, -1, 1, -1): 1}
    op.matrix = allocate_matrix(mss, msd, mds, op_channels)
    op.misc["pwd type"] = "analytic"
    op.check()
    return op

In [ ]:
def convolute(data):
    filename = data["file"]
    dens = read_h5(os.path.join(DENS_DIR, filename))
    p = dens.p
    q = dens.current_info["qval"] + 1.0e-7
    cutoff = data["Lam"] / HBARC

    fcn_ss = ExpressionMap(ss.subs({"L": cutoff, "k": q}), ["p_o", "p_i"])
    fcn_sd = ExpressionMap(sd.subs({"L": cutoff, "k": q}), ["p_o", "p_i"])
    fcn_ds = ExpressionMap(ds.subs({"L": cutoff, "k": q}), ["p_o", "p_i"])

    matrix_ss = fcn_ss(p, p)
    matrix_sd = fcn_sd(p, p)
    matrix_ds = fcn_ds(p, p)

    op = allocate_op(matrix_ss, matrix_sd, matrix_ds, op_channels, p)
    op.misc["Lam"] = data["Lam"]
    op.misc["q"] = q

    idx_dens, idx_op = get_channel_overlap_indices(dens.channels, op.channels)
    p2wp = dens.p ** 2 * dens.wp
    weight = p2wp.reshape(-1, 1) * p2wp

    return np.sum(dens.matrix[idx_dens] * op.matrix[idx_op] * weight)

In [ ]:
result = densities.copy()
result["q"] = result["omega"] / HBARC * 2
result["nme"] = result.apply(convolute, axis=1)

## Present result

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt

In [ ]:
densities.drop(columns=["omega", "file"]).describe(include="O").T

In [ ]:
densities.drop(columns=["omega", "file"]).describe(
    include=[np.number], percentiles=[]
).T[["mean"]]

In [ ]:
fig, ax = plt.subplots(figsize=(3, 2), dpi=200)

result[["q", "nme"]].plot(
    x="q", y="nme", ax=ax, ls="--", lw=1, marker="o", ms=4, legend=False
)

ax.set_ylabel(r"$\left\langle O_{2N}(\Lambda) \right\rangle$")
ax.set_xlabel(r"$q$ [fm$^{-1}$]")

sns.despine()

plt.show()

In [ ]:
print(result[["q", "nme"]])

In [ ]:
fig.savefig("2ncontact.pdf", bbox_inches="tight")